<a href="https://colab.research.google.com/github/RamshaB/AI/blob/main/HF_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Libraries Required
!pip install langchain-huggingface
## For API Calls
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
sec_key = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:

from langchain_huggingface import HuggingFaceEndpoint
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=sec_key



In [ ]:

from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_id="gpt2"
model=AutoModelForCausalLM.from_pretrained(model_id)
tokenizer=AutoTokenizer.from_pretrained(model_id)

In [ ]:
pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=100)
hf=HuggingFacePipeline(pipeline=pipe)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
hf

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ee81594ff70>, model_id='gpt2')

In [ ]:
hf.invoke("Google is a company with capital")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Google is a company with capitalized assets of more than $150 billion and owns the online video marketplace ZOOM.\n\nZOOM recently announced a $500 million plan for the company and is building a pilot program that is slated to take 1,000 employees to Boston.\n\nZOOM was awarded a $9.4 billion bid for the business by Nasdaq. With a market capitalization of $80 billion (the top five markets for digital video are Wall Street, CBS, Twitter, and Yahoo),'

In [ ]:
## Use HuggingfacePipelines With Gpu

from langchain_huggingface import HuggingFacePipeline
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 100},
)


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

In [ ]:
chain=prompt|gpu_llm


In [ ]:
question="What is artificial intelligence?"
chain.invoke({"question":question})

"Question: What is artificial intelligence?\n\nAnswer: Let's think step by step. You can imagine an agent learning how to play. What's a good actor learning how to play a game and being a hero? Who is the main character in his game, who needs a lot of help or something. An actor playing a game learning how to play, like a computer playing chess, will probably have a lot more training in how to play in an online simulator than an actor playing the game on video. The game has a lot more training because one has to learn how to play."